In [1]:
# 필요 라이브러리 로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#출력 그림 크기 조절
sns.set(rc={'figure.figsize':(12,12)})
plt.style.use('ggplot')
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
import warnings
warnings.filterwarnings("ignore")
import datetime as dt

### 세종사전 실행
from konlpy.tag import Twitter
from collections import Counter
from wordcloud import WordCloud
twitter = Twitter()

In [2]:
today = '0917'

In [3]:
df_sns = pd.read_csv("sns데이터/sns통합.csv",encoding = 'cp949')
df_sns

,아이디,날짜,내용
0,@Th0masm0re,2017-10-10,청와대 국민소통 광장 국민청원
1,@_iScent_,2017-10-10,청와대 국민청원
2,@xiu2_xiu2,2017-10-10,아 저 국민청원
3,@ansun9788,2017-10-10,청와대 국민소통 광장 국민청원 국민청원 이명박 제조사 원합니다
4,@s17221,2017-10-10,일베싸이트 폐지 국민청원
...,...,...,...
93639,@cpflrhkddl1,2021-01-01,자가격리 윤일병사망사건 안예은 국방부 일상 청소년 군대군인 강남구 청년 본부장 손수...
93640,@SXct6ccVE2kAQZt,2021-01-01,청와대 국민청원
93641,@SXct6ccVE2kAQZt,2021-01-01,청와대 국민청원
93642,@dgtwt,2021-01-01,코로나 걸려 죽기 전에 굶어 죽는다 자영업자 국민청원 건 육박


In [4]:
df = pd.read_csv("blue_text_0811_13000s.csv",encoding = 'cp949')
df

,Num,Class,Title,Date,Support
0,454161,안전/환경,텔레그램 n번방 용의자 신상공개 및 포토라인 세워주세요,2020-04-17,2715626
1,454160,안전/환경,텔레그램 n번방 가입자 전원의 신상공개를 원합니다,2020-04-19,2026252
2,454159,정치개혁,자유 한국당 정당해산 청원,2019-05-22,1831900
3,454158,기타,문재인 대통령님을 응원 합니다!,2020-03-27,1504597
4,454157,정치개혁,문재인 대통령 탄핵을 촉구합니다.,2020-03-05,1469023
...,...,...,...,...,...
78846,363287,인권/성평등,용산참사 재조사 및 관련자 처벌,2018-02-25,25
78847,363286,교통/건축/국토,"집값 상승의 폭탄, 개포8 단지 분양에 실거주 의무기간을 삽입해주세요",2018-03-28,25
78848,363285,문화/예술/체육/언론,"라이브 뷰잉에서 믹스, 이엣타이가 등을 넣는 사람들을 엄중 처벌 해주세요",2018-03-28,25
78849,363284,행정,정형식 판사 파면을 촉구합니다.,2018-03-08,25


In [59]:
def word_cloud(text,r,cs):

    ### 단어리스트 만들기
    word_list= text.to_list()
    ### 형태소 분리
    sentences_tag = []

    for sentence in word_list:
       morph = twitter.pos(sentence)
       sentences_tag.append(morph)
    
    ## 명사만 추출
    noun_list = []
    for sentence in sentences_tag:
       for word, tag in sentence:
           if tag in ['Noun']:
               noun_list.append(word)
                
    ## 두 단어 이상만 추출
    len(noun_list)

    noun_list = [n for n in noun_list if len(n) > 1]
    noun_list[:100]
    
    # 불용어 제거
    stopwords = ["청원", '처벌', '대한', '요청', '반대', '국민', '사건', '조사', '촉구', '요구', '불법', '대한민국', '개선', '관련'
             ,'한국','청와대',"뉴스","대한민국",]## 뉴스 제거


    noun_list = [i for i in noun_list if i not in stopwords]
    #print(word_list)
    counts = Counter(noun_list)
    tags = counts.most_common(30)

    
    ### WordCloud 생성
#     wordcloud = WordCloud(font_path='C:/Windows/Fonts/malgun.ttf',
#                       background_color='white',
#                       width=800,
#                       height=600)

    df_tag = pd.DataFrame(data = tags)
#    df_tag.to_csv("relation/{}_{}.csv".format(cs.split("/")[0],r),index = False, encoding = "cp949")
    display(df_tag[:15].T)
    display(df_tag[15:].T)
    print()
    
#     cloud = wordcloud.generate_from_frequencies(dict(tags))
#     plt.figure(figsize=(10, 8))
#     plt.axis('off')
#     plt.imshow(cloud)
#     plt.show()
    
#     df_tag = pd.DataFrame(data = new_data)
    df_tag.loc[:,"Source"] = 'SNS {}기간'.format(r)
#    display(df_new)
    return df_tag

In [60]:
def wc_0908(df1,df2,cs):
    ### 분류명 cs의 top 분류 선정
    temp = df1.loc[df1.Class == cs]
    temp = temp.loc[temp.Support == temp.Support.max()]
    display(temp)
    print()
    
    ddd = "날짜"
    ttt = "내용"
    
    word_r = pd.DataFrame()
    ## 기간 설정
    temp.Date = pd.to_datetime(temp.Date)
    df2[ddd] = pd.to_datetime(df2[ddd])
    starta = temp.Date -  dt.timedelta(days=61)
    startb = temp.Date - dt.timedelta(days=31)
    startc = temp.Date
    finishc = temp.Date + dt.timedelta(days=31)
    
    display(starta,startb,startc,finishc)
    print()
    
    df_A = df2.loc[df2[ddd].values >= starta.values]
    df_A = df_A.loc[df_A[ddd].values <=startb.values]
    print(cs, "분야의 청원 작성일 -1일 부터 -30일까지(기간A) {}건 워드클라우드".format(df_A[ttt].count()))
    print()
    if df_A[ttt].count() != 0:
        temp_A = word_cloud(df_A[ttt],'A',cs)
    else :
        temp_A = pd.DataFrame(columns=["Source","Target","Weight"])
    
    print()
    df_B = df2.loc[df2[ddd].values>startb.values]
    df_B = df_B.loc[df_B[ddd].values <=startc.values]
    print(cs, "분야의 청원 작성일 당일 부터 +30일까지(기간B) {}건 워드클라우드".format(df_B[ttt].count()))
    print()
    if df_B[ttt].count() != 0:
        temp_B = word_cloud(df_B[ttt],'B',cs)
    else :
        temp_B = pd.DataFrame(columns=["Source","Target","Weight"])
    
    print()
    df_C = df2.loc[df2[ddd].values>startc.values]
    df_C = df_C.loc[df_C[ddd].values <=finishc.values]
    print(cs, "분야의 청원 완료일 +1일 부터 +30일까지(기간C) {}건 워드클라우드".format(df_C[ttt].count()))
    print()
    if df_C[ttt].count() != 0:
        temp_C = word_cloud(df_C[ttt],'C',cs)
    else :
        temp_C = pd.DataFrame(columns=["Source","Target","Weight"])
        
    temp_All = pd.concat([temp_A,temp_B,temp_C],axis=0)
    #temp_All = temp_All.loc[:,["Source",0]]
    temp_All = temp_All.rename(columns={0 : "Target",1:"Weight"})
    display(temp_All)
#   temp_All.단어 = temp_All.apply(lambda x : x["단어"].split("\'")[1],axis=1)
    temp_All.to_csv("relation/sns_{}_{}_30.csv".format(cs.split("/")[0],today),index = False, encoding = "cp949")
    temp_All = temp_All.loc[:,["Source","Target","Weight"]]
    return temp_All

In [7]:
# 경제민주화
temp_1 = wc_0908(df,df_sns,"경제민주화")

,Num,Class,Title,Date,Support
136,454025,경제민주화,삼성증권 시스템 규제와 공매도 금지,2018-05-06,242286


136   2018-03-06
Name: Date, dtype: datetime64[ns]

136   2018-04-05
Name: Date, dtype: datetime64[ns]

136   2018-05-06
Name: Date, dtype: datetime64[ns]

136   2018-06-06
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,종편,폭행,전달,대위,파면,동의,광주,김기식,증권,청소년,개정,폐지,논란,등장,공매도
1,154,153,146,144,139,138,137,133,133,132,131,128,116,115,114




경제민주화 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 2935건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,게시판,대통령,수지,소통,광장,문재인,다음,중단,양예원,동의,판사,수사,논란,국회,매크로
1,282,225,222,211,192,148,140,132,130,125,120,118,115,114,109


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,경호,근황,비서,전두환,네이버,파면,한나라당,전달,출처,경찰,나경원,노태우,지지,폐지,공개
1,109,109,103,97,95,95,92,92,88,87,87,84,84,83,82


,Target,Weight,Source
0,장자연,459,SNS A기간
1,돌파,420,SNS A기간
2,여옥,318,SNS A기간
3,다음,309,SNS A기간
4,소통,288,SNS A기간
...,...,...,...
25,나경원,87,SNS C기간
26,노태우,84,SNS C기간
27,지지,84,SNS C기간
28,폐지,83,SNS C기간


In [8]:
# 경제민주화
temp_2 =wc_0908(df,df_sns,"안전/환경")

,Num,Class,Title,Date,Support
0,454161,안전/환경,텔레그램 n번방 용의자 신상공개 및 포토라인 세워주세요,2020-04-17,2715626


0   2020-02-16
Name: Date, dtype: datetime64[ns]

0   2020-03-17
Name: Date, dtype: datetime64[ns]

0   2020-04-17
Name: Date, dtype: datetime64[ns]

0   2020-05-18
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,오덕,박사,하루,출처,참여,재판,극복,네이버,사망,미국,중국,확진,독도,진단,민족
1,189,183,157,147,142,142,142,141,139,123,111,105,103,101,101




안전/환경 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1532건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,동의,답변,승인,장애인,채널,조선,취소,알맹이,대통령,사설,교사,단독,출처,달라,다음
1,126,119,98,97,86,85,81,79,78,77,74,73,68,65,63


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,돌파,파면,투표,등교,네이버,사람,아파트,피해자,우리,부정선거,누가,보도,명의,개학,재난
1,62,61,60,59,59,58,56,56,55,52,50,49,49,47,47


,Target,Weight,Source
0,문재인,1257,SNS A기간
1,대통령,1141,SNS A기간
2,신천지,1042,SNS A기간
3,탄핵,898,SNS A기간
4,대구,699,SNS A기간
...,...,...,...
25,누가,50,SNS C기간
26,보도,49,SNS C기간
27,명의,49,SNS C기간
28,개학,47,SNS C기간


In [9]:
# 기타
temp_3 =wc_0908(df,df_sns,"기타")

,Num,Class,Title,Date,Support
3,454158,기타,문재인 대통령님을 응원 합니다!,2020-03-27,1504597


3   2020-01-26
Name: Date, dtype: datetime64[ns]

3   2020-02-25
Name: Date, dtype: datetime64[ns]

3   2020-03-27
Name: Date, dtype: datetime64[ns]

3   2020-04-27
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,진자,사태,경북,다음,사망,중국,세계,국회,텔레,그램,조작,신상,검사,중국인,미국
1,306,303,289,285,275,273,238,223,221,221,211,210,207,202,201




기타 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1903건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,번방,동의,판사,승인,다음,오덕,답변,채널,조선,취소,코로나,국회,출처,재판,돌파
1,251,204,193,147,145,139,135,128,120,119,118,101,101,99,96


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,장애인,네이버,알맹이,세월호,하루,사설,총선,교체,대구시,수사,피해자,개정,우리,서명,검찰
1,90,88,78,77,77,76,76,70,65,64,64,61,60,59,59


,Target,Weight,Source
0,금지,289,SNS A기간
1,입국,286,SNS A기간
2,중국인,276,SNS A기간
3,신천지,261,SNS A기간
4,해체,169,SNS A기간
...,...,...,...
25,피해자,64,SNS C기간
26,개정,61,SNS C기간
27,우리,60,SNS C기간
28,서명,59,SNS C기간


In [10]:
# 문화/예술/체육/언론
temp_4 =wc_0908(df,df_sns,"문화/예술/체육/언론")

,Num,Class,Title,Date,Support
17,454144,문화/예술/체육/언론,"김보름, 박지우 선수의 자격박탈과 적폐 빙상연맹의 엄중 처벌을 청원합니다",2018-03-21,614127


17   2018-01-19
Name: Date, dtype: datetime64[ns]

17   2018-02-18
Name: Date, dtype: datetime64[ns]

17   2018-03-21
Name: Date, dtype: datetime64[ns]

17   2018-04-21
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,시간,추월,천안함,최단,일베,우롱,문재인,하나,다음,여자,라며,동료,개판,답변,게시판
1,290,282,269,246,229,197,190,189,176,174,170,168,167,165,165




문화/예술/체육/언론 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 3768건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,돌파,다음,대위,여옥,장자연,국회의원,징계,위증,소통,광장,전수,세월호,청문회,출처,동의
1,549,420,401,400,395,300,291,278,246,239,226,222,219,186,174


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,네이버,규명,죽음,참여,삼성,수사,청소년,증권,진실,폐지,표시,김기식,개정,동참,공매도
1,172,166,163,154,145,139,131,130,130,128,127,125,122,118,117


,Target,Weight,Source
0,파면,650,SNS A기간
1,강제,636,SNS A기간
2,개종,616,SNS A기간
3,답변,615,SNS A기간
4,판사,613,SNS A기간
...,...,...,...
25,표시,127,SNS C기간
26,김기식,125,SNS C기간
27,개정,122,SNS C기간
28,동참,118,SNS C기간


In [11]:
# 미래
temp_5 =wc_0908(df,df_sns,"미래")

,Num,Class,Title,Date,Support
16,454145,미래,조두순 출소반대,2017-12-05,615354


16   2017-10-05
Name: Date, dtype: datetime64[ns]

16   2017-11-04
Name: Date, dtype: datetime64[ns]

16   2017-12-05
Name: Date, dtype: datetime64[ns]

16   2018-01-05
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,정치,박근혜,참여,동의,조두순,접속,절대로,혼돈,폐가,붙이,이명,유야무야,요령,국정,외상
1,122,121,115,108,107,107,105,104,104,104,104,104,104,102,102




미래 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1130건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,소통,광장,문재인,참여,동의,유시민,폐지,답변,조두순,다음,동참,생각,게시판,대통령,소방관
1,117,106,98,90,65,60,54,51,47,45,44,43,40,39,39


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,탄핵,전안,경계,제도,자단,서명,교실,출처,네이버,작가,레진,사람,제발,불합리,조정
1,38,38,36,36,36,35,35,34,34,33,33,32,32,32,31


,Target,Weight,Source
0,동의,749,SNS A기간
1,요령,368,SNS A기간
2,접속,368,SNS A기간
3,촛불,257,SNS A기간
4,이명박,250,SNS A기간
...,...,...,...
25,레진,33,SNS C기간
26,사람,32,SNS C기간
27,제발,32,SNS C기간
28,불합리,32,SNS C기간


In [12]:
# 반려동물
temp_6 =wc_0908(df,df_sns,"반려동물")

,Num,Class,Title,Date,Support
98,454063,반려동물,고양이를 잔혹하게 학대하고 먹는 단체 오픈카톡방 “*****”을 수사하고 처벌하여 ...,2021-02-06,275492


98   2020-12-07
Name: Date, dtype: datetime64[ns]

98   2021-01-06
Name: Date, dtype: datetime64[ns]

98   2021-02-06
Name: Date, dtype: datetime64[ns]

98   2021-03-09
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,페이크,다음,계정은,사회,사람,사망,매우,디지털,등장,묵살,성추행,위원회,성폭력,수사,의사
1,68,66,60,58,56,54,54,53,52,49,49,48,47,47,47




반려동물 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1129건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,동의,장애인,대통령,링크,묵살,공유,청년,대학생,조작,다른,다음,긴급,단체,정부,기도
1,228,147,124,123,100,81,77,76,76,75,73,72,70,70,69


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,윤정희,검찰,도록,사망,죽음,계정,사회,등장,국가,강력범죄,프랑스,여러분,개혁,디지털,부디
1,65,58,55,52,52,49,49,49,48,48,46,45,42,42,42


,Target,Weight,Source
0,재판,350,SNS A기간
1,탄핵,309,SNS A기간
2,정경,251,SNS A기간
3,검찰,204,SNS A기간
4,동의,201,SNS A기간
...,...,...,...
25,프랑스,46,SNS C기간
26,여러분,45,SNS C기간
27,개혁,42,SNS C기간
28,디지털,42,SNS C기간


In [13]:
# 보건복지
temp_7 =wc_0908(df,df_sns,"보건복지")

,Num,Class,Title,Date,Support
8,454153,보건복지,중국인 입국 금지 요청,2020-02-22,761833


8   2019-12-23
Name: Date, dtype: datetime64[ns]

8   2020-01-22
Name: Date, dtype: datetime64[ns]

8   2020-02-22
Name: Date, dtype: datetime64[ns]

8   2020-03-24
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,번방,대구,신종,진짜,문재인,취소,국회,가해자,집회,정부,사람,수사,진자,대통령,해체
1,26,25,25,21,21,21,19,18,18,17,17,17,17,16,15




보건복지 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 4511건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,문재인,대통령,신천지,탄핵,응원,코로나,대구,정부,돌파,번방,공개,출처,경북,네이버,동의
1,1763,1501,1047,908,871,851,696,531,471,457,363,349,339,328,327


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,해체,확진,사태,진자,사망,중국,다음,세계,텔레,그램,조작,중국인,국회,검사,신상
1,325,313,301,292,265,257,252,221,209,209,209,209,202,202,189


,Target,Weight,Source
0,전광훈,1445,SNS A기간
1,한기총,1362,SNS A기간
2,구속,1319,SNS A기간
3,해산,1286,SNS A기간
4,수사,212,SNS A기간
...,...,...,...
25,조작,209,SNS C기간
26,중국인,209,SNS C기간
27,국회,202,SNS C기간
28,검사,202,SNS C기간


In [14]:
# 성장동력
temp_8 =wc_0908(df,df_sns,"성장동력")

,Num,Class,Title,Date,Support
121,454040,성장동력,전안법 합리적으로 개정 또는 폐지해주세요.(소상공인 소비자 모두 죽는법안입니다.),2018-01-02,255554


121   2017-11-02
Name: Date, dtype: datetime64[ns]

121   2017-12-02
Name: Date, dtype: datetime64[ns]

121   2018-01-02
Name: Date, dtype: datetime64[ns]

121   2018-02-02
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,동참,게시판,감형,국회,경계,대통령,전안,사람,교실,진짜,불합리,조정,논란,레진,작가
1,44,42,40,40,39,38,38,35,35,34,34,34,34,34,33




성장동력 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 2714건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,강제,개종,답변,육박,기록,증발,파장,신상,나경원,파면,단독,위원,소통,화폐,광장
1,570,558,509,466,448,442,438,437,431,331,294,212,200,200,194


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,가상,천지일보,여상규,돌파,네이버,조직,규제,다음,정부,출처,의원,평창올림픽,검사,국회의원,등장
1,180,162,156,155,132,129,127,127,112,110,110,108,105,97,95


,Target,Weight,Source
0,이명박,387,SNS A기간
1,소통,345,SNS A기간
2,대통령,315,SNS A기간
3,출국금지,275,SNS A기간
4,광장,242,SNS A기간
...,...,...,...
25,의원,110,SNS C기간
26,평창올림픽,108,SNS C기간
27,검사,105,SNS C기간
28,국회의원,97,SNS C기간


In [15]:
# 안전/환경
temp_9 =wc_0908(df,df_sns,"안전/환경")

,Num,Class,Title,Date,Support
0,454161,안전/환경,텔레그램 n번방 용의자 신상공개 및 포토라인 세워주세요,2020-04-17,2715626


0   2020-02-16
Name: Date, dtype: datetime64[ns]

0   2020-03-17
Name: Date, dtype: datetime64[ns]

0   2020-04-17
Name: Date, dtype: datetime64[ns]

0   2020-05-18
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,오덕,박사,하루,출처,참여,재판,극복,네이버,사망,미국,중국,확진,독도,진단,민족
1,189,183,157,147,142,142,142,141,139,123,111,105,103,101,101




안전/환경 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1532건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,동의,답변,승인,장애인,채널,조선,취소,알맹이,대통령,사설,교사,단독,출처,달라,다음
1,126,119,98,97,86,85,81,79,78,77,74,73,68,65,63


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,돌파,파면,투표,등교,네이버,사람,아파트,피해자,우리,부정선거,누가,보도,명의,개학,재난
1,62,61,60,59,59,58,56,56,55,52,50,49,49,47,47


,Target,Weight,Source
0,문재인,1257,SNS A기간
1,대통령,1141,SNS A기간
2,신천지,1042,SNS A기간
3,탄핵,898,SNS A기간
4,대구,699,SNS A기간
...,...,...,...
25,누가,50,SNS C기간
26,보도,49,SNS C기간
27,명의,49,SNS C기간
28,개학,47,SNS C기간


In [16]:
# 외교/통일/국방
temp_10 =wc_0908(df,df_sns,"외교/통일/국방")

,Num,Class,Title,Date,Support
12,454149,외교/통일/국방,"제주도 불법 난민 신청 문제에 따른 난민법, 무사증 입국, 난민신청허가 폐지/개헌 ...",2018-07-13,714875


12   2018-05-13
Name: Date, dtype: datetime64[ns]

12   2018-06-12
Name: Date, dtype: datetime64[ns]

12   2018-07-13
Name: Date, dtype: datetime64[ns]

12   2018-08-13
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,조직,이용,정보,양승태,몰래,불분명,조선시대,지주,적폐,협회,대통령,논란,고양이,다음,출처
1,113,112,111,109,109,109,107,107,95,88,88,85,81,81,81




외교/통일/국방 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1627건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,이재명,조폭,은수미,폐지,소통,광장,의혹,연루,참여,대법원,게시판,답변,출처,네이버,국민연금
1,398,200,162,161,147,147,139,138,138,128,119,98,98,97,92


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,난민,유착,나라,다음,시작,현재,누진,인원,수사,규명,진상,조직,이용,강탈,양승태
1,90,89,86,84,84,82,81,80,78,74,74,72,68,67,66


,Target,Weight,Source
0,게시판,267,SNS A기간
1,대통령,234,SNS A기간
2,수지,231,SNS A기간
3,소통,188,SNS A기간
4,광장,174,SNS A기간
...,...,...,...
25,진상,74,SNS C기간
26,조직,72,SNS C기간
27,이용,68,SNS C기간
28,강탈,67,SNS C기간


In [17]:
# 육아/교육
temp_11 =wc_0908(df,df_sns,"육아/교육")

,Num,Class,Title,Date,Support
20,454141,육아/교육,저희 25개월딸이 초등학생 5학년에게 성폭행을 당했습니다,2020-04-19,533883


20   2020-02-18
Name: Date, dtype: datetime64[ns]

20   2020-03-19
Name: Date, dtype: datetime64[ns]

20   2020-04-19
Name: Date, dtype: datetime64[ns]

20   2020-05-20
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,오덕,박사,하루,출처,재판,참여,사망,네이버,미국,중국,승인,확진,극복,독도,진단
1,189,177,161,144,144,143,139,133,113,109,106,105,104,103,101




육아/교육 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1636건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,동의,답변,출처,장애인,대통령,알맹이,네이버,사설,개월,교사,사람,단독,달라,등교,다음
1,141,123,85,84,79,79,77,77,75,74,73,73,71,65,63


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,성폭행,거짓,파면,투표,우리,재난,진짜,아파트,피해자,보도,명의,부정선거,원금,누가,개학
1,62,61,61,60,56,55,53,53,53,52,50,49,48,48,47


,Target,Weight,Source
0,문재인,1275,SNS A기간
1,대통령,1158,SNS A기간
2,신천지,1046,SNS A기간
3,탄핵,899,SNS A기간
4,코로나,717,SNS A기간
...,...,...,...
25,명의,50,SNS C기간
26,부정선거,49,SNS C기간
27,원금,48,SNS C기간
28,누가,48,SNS C기간


In [18]:
# 인권/성평등
temp_12 =wc_0908(df,df_sns,"인권/성평등")

,Num,Class,Title,Date,Support
5,454156,인권/성평등,"신천지 예수교 증거장막성전(이하, 신천지)의 강제 해체(해산)을 청원합니다.",2020-03-23,1449521


5   2020-01-22
Name: Date, dtype: datetime64[ns]

5   2020-02-21
Name: Date, dtype: datetime64[ns]

5   2020-03-23
Name: Date, dtype: datetime64[ns]

5   2020-04-23
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,사태,동의,진자,확진,중국,다음,사망,중국인,세계,조작,금지,검사,텔레,그램,입국
1,299,295,281,280,247,241,231,220,219,206,203,200,195,195,190




인권/성평등 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 2331건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,번방,판사,동의,문재인,코로나,대통령,오덕,다음,국회,승인,재판,정부,돌파,하루,채널
1,417,280,253,236,227,189,189,185,155,148,143,141,139,128,128


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,출처,조선,취소,응원,네이버,독도,키트,진단,공개,총선,답변,참여,교체,개정,신상
1,126,121,121,117,113,104,101,96,96,93,86,86,85,79,77


,Target,Weight,Source
0,금지,245,SNS A기간
1,입국,242,SNS A기간
2,중국인,235,SNS A기간
3,사회,67,SNS A기간
4,문화,65,SNS A기간
...,...,...,...
25,답변,86,SNS C기간
26,참여,86,SNS C기간
27,교체,85,SNS C기간
28,개정,79,SNS C기간


In [19]:
# 일자리
temp_13 =wc_0908(df,df_sns,"일자리")

,Num,Class,Title,Date,Support
49,454112,일자리,한전 사업에 중국 기업의 참여를 허락하는 것은 말도 안됩니다.,2020-03-27,383039


49   2020-01-26
Name: Date, dtype: datetime64[ns]

49   2020-02-25
Name: Date, dtype: datetime64[ns]

49   2020-03-27
Name: Date, dtype: datetime64[ns]

49   2020-04-27
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,진자,사태,경북,다음,사망,중국,세계,국회,텔레,그램,조작,신상,검사,중국인,미국
1,306,303,289,285,275,273,238,223,221,221,211,210,207,202,201




일자리 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1903건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,번방,동의,판사,승인,다음,오덕,답변,채널,조선,취소,코로나,국회,출처,재판,돌파
1,251,204,193,147,145,139,135,128,120,119,118,101,101,99,96


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,장애인,네이버,알맹이,세월호,하루,사설,총선,교체,대구시,수사,피해자,개정,우리,서명,검찰
1,90,88,78,77,77,76,76,70,65,64,64,61,60,59,59


,Target,Weight,Source
0,금지,289,SNS A기간
1,입국,286,SNS A기간
2,중국인,276,SNS A기간
3,신천지,261,SNS A기간
4,해체,169,SNS A기간
...,...,...,...
25,피해자,64,SNS C기간
26,개정,61,SNS C기간
27,우리,60,SNS C기간
28,서명,59,SNS C기간


In [20]:
# 정치개혁
temp_14 =wc_0908(df,df_sns,"정치개혁")

,Num,Class,Title,Date,Support
2,454159,정치개혁,자유 한국당 정당해산 청원,2019-05-22,1831900


2   2019-03-22
Name: Date, dtype: datetime64[ns]

2   2019-04-21
Name: Date, dtype: datetime64[ns]

2   2019-05-22
Name: Date, dtype: datetime64[ns]

2   2019-06-22
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,출처,네이버,왜구,촛불,접속,참여,국회,토착,자유당,역대,서명,북한,현재,게시판,베트남
1,331,330,327,324,318,309,308,301,300,293,292,280,277,270,266




정치개혁 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1279건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,답변,대통령,해산,하야,탄핵,다음,문재인,등장,국회,김원봉,의원,한기총,동의,효상,발언
1,188,157,119,92,91,90,90,81,78,77,72,71,68,68,65


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,자유,출처,정당,네이버,수사,돌파,국민소환제,참여,국회의원,성폭행,게시판,신영복,민주당,판사,정부
1,64,64,63,61,56,55,52,50,50,49,48,48,42,42,41


,Target,Weight,Source
0,세월호,339,SNS A기간
1,설치,234,SNS A기간
2,특별,211,SNS A기간
3,참사,205,SNS A기간
4,윤지오,198,SNS A기간
...,...,...,...
25,게시판,48,SNS C기간
26,신영복,48,SNS C기간
27,민주당,42,SNS C기간
28,판사,42,SNS C기간


In [21]:
# 행정
temp_14 = wc_0908(df,df_sns,"행정")

,Num,Class,Title,Date,Support
34,454127,행정,정부의 교회 정규 예배 이외 행사 금지를 취소해주세요.,2020-08-07,427470


34   2020-06-07
Name: Date, dtype: datetime64[ns]

34   2020-07-07
Name: Date, dtype: datetime64[ns]

34   2020-08-07
Name: Date, dtype: datetime64[ns]

34   2020-09-07
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,현재,문화,등장,행사,예술의전당,예배,앵커,하차,대통령,피해자,이소정,정규,이외,우리,호소
1,57,55,53,52,48,48,47,45,40,39,39,39,38,37,37




행정 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 3085건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,집회,탄핵,추미애,광화문,장관,동의,다음,출처,네이버,판사,의사,돌파,일당,시무,코로나
1,683,664,656,555,521,436,424,325,296,277,272,266,231,228,216


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,전광훈,의대,해임,참가자,파업,보도,교회,답변,파문,취소,소문,대통령,사랑,의료,허용
1,212,209,208,191,190,185,180,170,160,157,148,147,147,144,138


,Target,Weight,Source
0,택시,116,SNS A기간
1,동의,110,SNS A기간
2,기사,104,SNS A기간
3,박병석,101,SNS A기간
4,판사,98,SNS A기간
...,...,...,...
25,소문,148,SNS C기간
26,대통령,147,SNS C기간
27,사랑,147,SNS C기간
28,의료,144,SNS C기간


In [40]:
# 교통
temp_교통 =wc_0908(df,df_sns,"교통/건축/국토")


,Num,Class,Title,Date,Support
13,454148,교통/건축/국토,강원도 차이나타운 건설을 철회해주세요.,2021-04-28,670780


13   2021-02-26
Name: Date, dtype: datetime64[ns]

13   2021-03-28
Name: Date, dtype: datetime64[ns]

13   2021-04-28
Name: Date, dtype: datetime64[ns]

13   2021-05-29
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,영초,해시태그,다음,제발,연예,묵살,차이나타운,방송,강력,특수,무죄,사법부,강원도,여성,수가
1,197,197,183,156,154,151,108,108,97,95,94,93,87,85,84




교통/건축/국토 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1305건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,동의,대학생,청년,접종,사망,백신,사람,출처,네이버,우리,자녀,국회의원,진짜,대통령,다음
1,110,97,84,76,70,69,64,60,58,55,53,52,52,51,51


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,무보수,명예,공천,수사,엄벌,피해자,증제,일병,달라,폐지,고함,군인,청소년,금융,국방부
1,50,50,49,49,49,48,46,46,46,45,43,42,42,42,42


,Target,Weight,Source
0,동의,219,SNS A기간
1,축구,204,SNS A기간
2,한일전,200,SNS A기간
3,장애인,175,SNS A기간
4,일본,173,SNS A기간
...,...,...,...
25,고함,43,SNS C기간
26,군인,42,SNS C기간
27,청소년,42,SNS C기간
28,금융,42,SNS C기간


In [61]:
# 농어촌
temp_농어촌 =wc_0908(df,df_sns,"농산어촌")

,Num,Class,Title,Date,Support
1069,453089,농산어촌,왜 어민이 농민보다 세금을 더 내야 합니까?,2019-09-01,28310


1069   2019-07-02
Name: Date, dtype: datetime64[ns]

1069   2019-08-01
Name: Date, dtype: datetime64[ns]

1069   2019-09-01
Name: Date, dtype: datetime64[ns]

1069   2019-10-02
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,노무현,동의,주지,노회찬,일본,다음,밀정,비공개,실검,장악,언론,위해,서명,배신,취소
1,146,142,141,141,141,137,133,116,112,108,106,103,93,92,92




농산어촌 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1842건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,검찰,조국,윤석열,개혁,돌파,임명,언론,나경원,장관,법무부,대통령,특검,의혹,두렁,시계
1,1122,921,689,613,588,582,580,574,522,443,359,351,340,299,299


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,재현,문재인,누설,기밀,가짜,열망,절실,더욱,역적,자유,다음,수사,아들,정보,서울대
1,278,267,250,249,220,197,195,181,175,174,173,168,166,128,128


,Target,Weight,Source
0,일본,402,SNS A기간
1,조선일보,374,SNS A기간
2,폐간,211,SNS A기간
3,신문,192,SNS A기간
4,유승준,122,SNS A기간
...,...,...,...
25,다음,173,SNS C기간
26,수사,168,SNS C기간
27,아들,166,SNS C기간
28,정보,128,SNS C기간


In [62]:
# 행정
temp_저출산 =wc_0908(df,df_sns,"저출산/고령화대책")

,Num,Class,Title,Date,Support
1136,453021,저출산/고령화대책,서울시임신출산정보센터 논란글 담당작성자와 책임자 징계 및 공개 사과를 요구합니다.,2021-02-05,26068


1136   2020-12-06
Name: Date, dtype: datetime64[ns]

1136   2021-01-05
Name: Date, dtype: datetime64[ns]

1136   2021-02-05
Name: Date, dtype: datetime64[ns]

1136   2021-03-08
Name: Date, dtype: datetime64[ns]

...

,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,링크,사회,사람,디지털,등장,사망,묵살,퍼트,성추행,계정은,위원회,성폭력,수사,지금,의사
1,58,58,55,52,52,50,49,49,49,48,47,47,47,47,47




저출산/고령화대책 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1143건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,동의,장애인,링크,대통령,묵살,공유,조작,다음,다른,윤정희,청년,긴급,대학생,단체,기도
1,228,144,143,123,99,92,75,74,74,74,71,71,70,69,68


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,계정,여러분,방치,프랑스,퍼트,도록,정부,사망,부디,죽음,등장,강력범죄,사회,출처,방법
1,67,65,62,62,60,55,53,52,52,52,49,48,46,44,40


,Target,Weight,Source
0,재판,350,SNS A기간
1,탄핵,304,SNS A기간
2,정경,251,SNS A기간
3,검찰,215,SNS A기간
4,추미애,196,SNS A기간
...,...,...,...
25,등장,49,SNS C기간
26,강력범죄,48,SNS C기간
27,사회,46,SNS C기간
28,출처,44,SNS C기간


In [63]:
def word_cloud(text,r,cs):
    
    ### 단어리스트 만들기
    word_list= text.to_list()
    ### 형태소 분리
    sentences_tag = []
    for i in word_list:
        k = i.split(",")
        for n in k:
            sentences_tag.append(n)
        
    counts = Counter(sentences_tag)
    tags = counts.most_common(30)

    
#     ### WordCloud 생성
#     wordcloud = WordCloud(font_path='C:/Windows/Fonts/malgun.ttf',
#                       background_color='white',
#                       width=800,
#                       height=600)
    
    df_tag = pd.DataFrame(data = tags)
#    df_tag.to_csv("relation/{}_{}.csv".format(cs.split("/")[0],r),index = False, encoding = "cp949")
    display(df_tag[:15].T)
    display(df_tag[15:].T)
    print()
    
#     cloud = wordcloud.generate_from_frequencies(dict(tags))
#     plt.figure(figsize=(10, 8))
#     plt.axis('off')
#     plt.imshow(cloud)
#     plt.show()
    
#     new_data = []
#     for i_1 in df_tag[0].unique():
#         for i_2 in range(int(df_tag.loc[df_tag[0] == i_1,1])):
#             new_data.append(i_1)
#     df_new = pd.DataFrame(data = new_data)
    df_tag.loc[:,"기간"] = '뉴스 {}기간'.format(r)
    display(df_tag)
    return df_tag

In [64]:
def wc_0908(df1,df2,cs):
    ### 분류명 cs의 top 분류 선정
    temp = df1.loc[df1.Class == cs]
    temp = temp.loc[temp.Support == temp.Support.max()]
    display(temp)
    print()
    
    word_r = pd.DataFrame()
    ## 기간 설정
    temp.Date = pd.to_datetime(temp.Date)
    df2["news Date"] = pd.to_datetime(df2["news Date"])
    starta = temp.Date -  dt.timedelta(days=61)
    startb = temp.Date - dt.timedelta(days=31)
    startc = temp.Date
    finishc = temp.Date + dt.timedelta(days=31)
    
    display(starta,startb,startc,finishc)
    print()
    
    df_A = df2.loc[df2["news Date"].values >= starta.values]
    df_A = df_A.loc[df_A["news Date"].values <=startb.values,["news Date","best keyword"]]
    print(cs, "분야의 청원 작성일 -1일 부터 -30일까지(기간A) {}건 워드클라우드".format(df_A["best keyword"].count()))
    print()
    if df_A["best keyword"].count() != 0:
        temp_A = word_cloud(df_A["best keyword"],'A',cs)
    else :
        temp_A = pd.DataFrame(columns=["Source","Target","Weight"])
    
    print()
    df_B = df2.loc[df2["news Date"].values>startb.values]
    df_B = df_B.loc[df_B["news Date"].values <=startc.values,["news Date","best keyword"]]
    print(cs, "분야의 청원 작성일 당일 부터 +30일까지(기간B) {}건 워드클라우드".format(df_B["best keyword"].count()))
    print()
    if df_B["best keyword"].count() != 0:
        temp_B = word_cloud(df_B["best keyword"],'B',cs)
    else :
        temp_B = pd.DataFrame(columns=["Source","Target","Weight"])
    
    print()
    df_C = df2.loc[df2["news Date"].values>startc.values]
    df_C = df_C.loc[df_C["news Date"].values <=finishc.values,["news Date","best keyword"]]
    print(cs, "분야의 청원 완료일 +1일 부터 +30일까지(기간C) {}건 워드클라우드".format(df_C["best keyword"].count()))
    print()
    if df_C["best keyword"].count() != 0:
        temp_C = word_cloud(df_C["best keyword"],'C',cs)
    else :
        temp_C = pd.DataFrame(columns=["Source","Target","Weight"])
        
    temp_All = pd.concat([temp_A,temp_B,temp_C],axis=0)
    temp_All = temp_All.loc[:,["기간",0,1]]
    temp_All = temp_All.rename(columns={"기간":"Source",0 : "Target",1:"Weight"})
    display(temp_All)
    #temp_All.to_csv("relation/{}_0917_30.csv".format(cs.split("/")[0]),index = False, encoding = "cp949")
    return temp_All

### 실행

-
-
-
-
-
-
-
-
-
-
-
-
-
-
-











In [24]:
# 경제민주화
df2 = pd.read_excel('news_0908/경제민주화_20180306-20180606.xlsx')
tm_1 = wc_0908(df,df2,"경제민주화")
t_1 = pd.concat([temp_1,tm_1])
t_1.to_csv("경제민주화_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
136,454025,경제민주화,삼성증권 시스템 규제와 공매도 금지,2018-05-06,242286


136   2018-03-06
Name: Date, dtype: datetime64[ns]

136   2018-04-05
Name: Date, dtype: datetime64[ns]

136   2018-05-06
Name: Date, dtype: datetime64[ns]

136   2018-06-06
Name: Date, dtype: datetime64[ns]

...


경제민주화 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 168건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,삼성증권,금감원,투자자,증권사,위원회,감독원,직원들,금융감독원,공매도,주식,임직원,삼성,관계자,계열사,가능성
1,134,59,53,43,40,38,36,35,33,30,30,30,27,26,23


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,미국,직원,금융위,한국,연구원,삼성전자,고의성,외국인,서울,금융위원회,배당,501만,팀장,중징계,코스피
1,20,20,20,19,19,19,19,17,15,15,15,14,14,13,13


,0,1,기간
0,삼성증권,134,뉴스 C기간
1,금감원,59,뉴스 C기간
2,투자자,53,뉴스 C기간
3,증권사,43,뉴스 C기간
4,위원회,40,뉴스 C기간
5,감독원,38,뉴스 C기간
6,직원들,36,뉴스 C기간
7,금융감독원,35,뉴스 C기간
8,공매도,33,뉴스 C기간
9,주식,30,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,삼성증권,64
1,뉴스 A기간,코스피,36
2,뉴스 A기간,미국,33
3,뉴스 A기간,코스닥,30
4,뉴스 A기간,외국인,23
...,...,...,...
25,뉴스 C기간,배당,15
26,뉴스 C기간,501만,14
27,뉴스 C기간,팀장,14
28,뉴스 C기간,중징계,13


In [25]:
# 기타
df2 = pd.read_excel('news_0908/기타_20200126-20200427.xlsx')
tm_2 = wc_0908(df,df2,"기타")
t_2 = pd.concat([temp_3,tm_2])
t_2.to_csv("기타_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
3,454158,기타,문재인 대통령님을 응원 합니다!,2020-03-27,1504597


3   2020-01-26
Name: Date, dtype: datetime64[ns]

3   2020-02-25
Name: Date, dtype: datetime64[ns]

3   2020-03-27
Name: Date, dtype: datetime64[ns]

3   2020-04-27
Name: Date, dtype: datetime64[ns]

...


기타 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 3210건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,문재인,민주당,코로나19,청와대,위원장,통합당,더불어민주당,서울,코로나,한국,미래통합당,코로,지원금,김종인,황교안
1,1722,1140,1058,793,732,644,635,634,473,470,412,400,376,349,340


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,지역구,미국,관계자,대통령,코로나바이러스,더불어시민당,유권자,이낙연,국회의원,정의당,국민들,부산,대한민국,일자리,긴급재난지원금
1,327,310,294,268,264,258,248,245,233,232,222,221,221,205,202


,0,1,기간
0,문재인,1722,뉴스 C기간
1,민주당,1140,뉴스 C기간
2,코로나19,1058,뉴스 C기간
3,청와대,793,뉴스 C기간
4,위원장,732,뉴스 C기간
5,통합당,644,뉴스 C기간
6,더불어민주당,635,뉴스 C기간
7,서울,634,뉴스 C기간
8,코로나,473,뉴스 C기간
9,한국,470,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,문재인,1496
1,뉴스 A기간,청와대,1005
2,뉴스 A기간,서울,606
3,뉴스 A기간,중국,539
4,뉴스 A기간,한국당,522
...,...,...,...
25,뉴스 C기간,국민들,222
26,뉴스 C기간,부산,221
27,뉴스 C기간,대한민국,221
28,뉴스 C기간,일자리,205


In [26]:
# 문화/예술/체육/언론
df2 = pd.read_excel('news_0908/문화_예술_체육_언론_20180119-20180421.xlsx')
tm_3 =wc_0908(df,df2,"문화/예술/체육/언론")
t_3 = pd.concat([temp_4,tm_3])
t_3.to_csv("문화_예술_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
17,454144,문화/예술/체육/언론,"김보름, 박지우 선수의 자격박탈과 적폐 빙상연맹의 엄중 처벌을 청원합니다",2018-03-21,614127


17   2018-01-19
Name: Date, dtype: datetime64[ns]

17   2018-02-18
Name: Date, dtype: datetime64[ns]

17   2018-03-21
Name: Date, dtype: datetime64[ns]

17   2018-04-21
Name: Date, dtype: datetime64[ns]

...


문화/예술/체육/언론 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 33건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,빙상연맹,평창,전명규,부회장,관계자,대한빙상경기연맹,스피드스케이팅,팀추월,노선영,금메달,청와대,연맹,논란,빙상,한국
1,23,22,19,16,13,13,13,11,11,10,9,9,9,8,8


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,청원,한국체대,노진규,이승훈,겨울왕국,김보름,sbs,올림픽,선수들,박지우,선수,게시판,사임서,매스,쇼트트랙
1,8,7,7,7,7,7,7,7,6,6,6,5,5,5,5


,0,1,기간
0,빙상연맹,23,뉴스 C기간
1,평창,22,뉴스 C기간
2,전명규,19,뉴스 C기간
3,부회장,16,뉴스 C기간
4,관계자,13,뉴스 C기간
5,대한빙상경기연맹,13,뉴스 C기간
6,스피드스케이팅,13,뉴스 C기간
7,팀추월,11,뉴스 C기간
8,노선영,11,뉴스 C기간
9,금메달,10,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,평창,82
1,뉴스 A기간,빙상연맹,63
2,뉴스 A기간,스피드스케이팅,56
3,뉴스 A기간,노선영,56
4,뉴스 A기간,한국,39
...,...,...,...
25,뉴스 C기간,선수,6
26,뉴스 C기간,게시판,5
27,뉴스 C기간,사임서,5
28,뉴스 C기간,매스,5


In [27]:
# 미래
df2 = pd.read_excel('news_0908/미래_20170806-20180105.xlsx')
tm_4 =wc_0908(df,df2,"미래")
t_4 = pd.concat([temp_5,tm_4])
t_4.to_csv("미래_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
16,454145,미래,조두순 출소반대,2017-12-05,615354


16   2017-10-05
Name: Date, dtype: datetime64[ns]

16   2017-11-04
Name: Date, dtype: datetime64[ns]

16   2017-12-05
Name: Date, dtype: datetime64[ns]

16   2018-01-05
Name: Date, dtype: datetime64[ns]

...


미래 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 57건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,조두순,청와대,청원,감경,불가능,성범죄,성폭행,주취감경,성폭력,피해자,무기징역,상해,범죄자,민정수석,홈페이지
1,54,37,35,27,23,22,19,18,14,14,12,12,11,11,11


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,심신미약,재심,전자발찌,피의자,개정안,음주,현행법,국민들,법무부,페이스북,수석,참여자,변호사,창원,서울
1,10,10,10,9,9,9,9,8,8,8,8,8,7,7,7


,0,1,기간
0,조두순,54,뉴스 C기간
1,청와대,37,뉴스 C기간
2,청원,35,뉴스 C기간
3,감경,27,뉴스 C기간
4,불가능,23,뉴스 C기간
5,성범죄,22,뉴스 C기간
6,성폭행,19,뉴스 C기간
7,주취감경,18,뉴스 C기간
8,성폭력,14,뉴스 C기간
9,피해자,14,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,조두순,7
1,뉴스 A기간,성폭행,6
2,뉴스 A기간,범죄자,6
3,뉴스 A기간,여중생,6
4,뉴스 A기간,변호사,6
...,...,...,...
25,뉴스 C기간,수석,8
26,뉴스 C기간,참여자,8
27,뉴스 C기간,변호사,7
28,뉴스 C기간,창원,7


In [28]:
# 반려동물
df2 = pd.read_excel('news_0908/반려동물_20201207-20210309.xlsx')
tm_5=wc_0908(df,df2,"반려동물")
t_5 = pd.concat([temp_6,tm_5])
t_5.to_csv("반려동물_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
98,454063,반려동물,고양이를 잔혹하게 학대하고 먹는 단체 오픈카톡방 “*****”을 수사하고 처벌하여 ...,2021-02-06,275492


98   2020-12-07
Name: Date, dtype: datetime64[ns]

98   2021-01-06
Name: Date, dtype: datetime64[ns]

98   2021-02-06
Name: Date, dtype: datetime64[ns]

98   2021-03-09
Name: Date, dtype: datetime64[ns]

...


반려동물 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 13건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,동물,동물원,마리,대구,동물들,과태료,대구시,코로나19,목줄,학대,네트워크,미이행,보호법,동물학대,운전자
1,6,6,5,5,5,5,5,4,4,4,4,4,3,3,3


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,경찰,확인,비글,행정처분,일본원숭이,일본,관계기관,반려동물,동물보호법,소유자,관계자,동물보호단체,대책,보호,계획
1,3,3,3,3,3,3,3,2,2,2,2,2,2,2,2


,0,1,기간
0,동물,6,뉴스 C기간
1,동물원,6,뉴스 C기간
2,마리,5,뉴스 C기간
3,대구,5,뉴스 C기간
4,동물들,5,뉴스 C기간
5,과태료,5,뉴스 C기간
6,대구시,5,뉴스 C기간
7,코로나19,4,뉴스 C기간
8,목줄,4,뉴스 C기간
9,학대,4,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,마리,11
1,뉴스 A기간,동물학대,10
2,뉴스 A기간,동물,5
3,뉴스 A기간,학대,5
4,뉴스 A기간,미국,5
...,...,...,...
25,뉴스 C기간,관계자,2
26,뉴스 C기간,동물보호단체,2
27,뉴스 C기간,대책,2
28,뉴스 C기간,보호,2


In [29]:
# 보건복지
df2 = pd.read_excel('news_0908/보건복지_20191223-20200324.xlsx')
tm_6 = wc_0908(df,df2,"보건복지")
t_6 = pd.concat([temp_7,tm_6])
t_6.to_csv("보건복지_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
8,454153,보건복지,중국인 입국 금지 요청,2020-02-22,761833


8   2019-12-23
Name: Date, dtype: datetime64[ns]

8   2020-01-22
Name: Date, dtype: datetime64[ns]

8   2020-02-22
Name: Date, dtype: datetime64[ns]

8   2020-03-24
Name: Date, dtype: datetime64[ns]

...


보건복지 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 213건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,중국,코로나19,한국,확진자,한국인,중국인,코로나,미국,일본,입국자,외국인,코로나바이러스,대구,입국금지,한국발
1,175,161,113,82,76,74,59,58,51,47,46,42,41,41,35


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,박능후,감염병,외교부,코로,청와대,강경화,이탈리아,중국발,신천지,문재인,유럽,감염증,사망자,전문가,경북
1,34,33,33,32,29,29,28,26,26,25,25,24,23,23,22


,0,1,기간
0,중국,175,뉴스 C기간
1,코로나19,161,뉴스 C기간
2,한국,113,뉴스 C기간
3,확진자,82,뉴스 C기간
4,한국인,76,뉴스 C기간
5,중국인,74,뉴스 C기간
6,코로나,59,뉴스 C기간
7,미국,58,뉴스 C기간
8,일본,51,뉴스 C기간
9,입국자,47,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,미국,2
1,뉴스 A기간,이슬람,2
2,뉴스 A기간,입국금지,2
3,뉴스 A기간,행정명령,2
4,뉴스 A기간,미얀마,2
...,...,...,...
25,뉴스 C기간,유럽,25
26,뉴스 C기간,감염증,24
27,뉴스 C기간,사망자,23
28,뉴스 C기간,전문가,23


In [58]:
# 성장동력
df2 = pd.read_excel('news_0908/성장동력_20171102-20180202.xlsx')
tm_7 =wc_0908(df,df2,"성장동력")
t_7 = pd.concat([temp_8,tm_7])
t_7.to_csv("성장동력_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
121,454040,성장동력,전안법 합리적으로 개정 또는 폐지해주세요.(소상공인 소비자 모두 죽는법안입니다.),2018-01-02,255554


121   2017-11-02
Name: Date, dtype: datetime64[ns]

121   2017-12-02
Name: Date, dtype: datetime64[ns]

121   2018-01-02
Name: Date, dtype: datetime64[ns]

121   2018-02-02
Name: Date, dtype: datetime64[ns]

...


성장동력 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 25건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,전안법,한국당,김종석,본회의,ㅅㄱㅂㅊ,국회의원,ㅁㅊㅅㄲ,청원,청와대,문자열,안전법,문자,조두순,20만,국민청원
1,17,15,13,12,12,11,11,10,10,9,9,8,7,7,7


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,의원,보호법,누리꾼,소상공인,나경원,전기용품,전기생활용품안전법,입장문,발신전용,건씩,문자메시지,시민,성폭행,평창,페이스북
1,7,6,6,5,5,5,5,5,5,5,5,5,4,4,4


,0,1,기간
0,전안법,17,뉴스 C기간
1,한국당,15,뉴스 C기간
2,김종석,13,뉴스 C기간
3,본회의,12,뉴스 C기간
4,ㅅㄱㅂㅊ,12,뉴스 C기간
5,국회의원,11,뉴스 C기간
6,ㅁㅊㅅㄲ,11,뉴스 C기간
7,청원,10,뉴스 C기간
8,청와대,10,뉴스 C기간
9,문자열,9,뉴스 C기간


,Source,Target,Weight
0,뉴스 B기간,전안법,18.0
1,뉴스 B기간,개정안,16.0
2,뉴스 B기간,본회의,15.0
3,뉴스 B기간,생활용품,12.0
4,뉴스 B기간,소상공인,12.0
5,뉴스 B기간,전기용품,9.0
6,뉴스 B기간,원내대표,9.0
7,뉴스 B기간,한국당,8.0
8,뉴스 B기간,감사원장,7.0
9,뉴스 B기간,후보자,7.0


In [31]:
# 안전/환경
df2 = pd.read_excel('news_0908/안전_환경_20200216-20200518.xlsx')
tm_8 =wc_0908(df,df2,"안전/환경")
t_8 = pd.concat([temp_9,tm_8])
t_8.to_csv("안전_환경_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
0,454161,안전/환경,텔레그램 n번방 용의자 신상공개 및 포토라인 세워주세요,2020-04-17,2715626


0   2020-02-16
Name: Date, dtype: datetime64[ns]

0   2020-03-17
Name: Date, dtype: datetime64[ns]

0   2020-04-17
Name: Date, dtype: datetime64[ns]

0   2020-05-18
Name: Date, dtype: datetime64[ns]

...


안전/환경 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 857건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,텔레그램,피해자,n번방,조주빈,박사방,성범죄,착취물,성착취,운영자,성착취물,갓갓,대화방,위원회,미성년자,영상물
1,487,305,267,242,232,228,195,189,186,161,160,158,144,138,137


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,성착,취물,피의자,음란물,성폭력,개정안,관계자,경찰청,본회의,민주당,경찰,서울,코로나19,구속영장,방지법
1,136,132,129,120,116,114,109,103,102,91,88,86,84,83,82


,0,1,기간
0,텔레그램,487,뉴스 C기간
1,피해자,305,뉴스 C기간
2,n번방,267,뉴스 C기간
3,조주빈,242,뉴스 C기간
4,박사방,232,뉴스 C기간
5,성범죄,228,뉴스 C기간
6,착취물,195,뉴스 C기간
7,성착취,189,뉴스 C기간
8,운영자,186,뉴스 C기간
9,성착취물,161,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,텔레그램,20
1,뉴스 A기간,코로나19,13
2,뉴스 A기간,성범죄,7
3,뉴스 A기간,미국,7
4,뉴스 A기간,청원,7
...,...,...,...
25,뉴스 C기간,경찰,88
26,뉴스 C기간,서울,86
27,뉴스 C기간,코로나19,84
28,뉴스 C기간,구속영장,83


In [32]:
# 외교/통일/국방
df2 = pd.read_excel('news_0908/외교_통일_국방_20180513-20180813.xlsx')
tm_9 =wc_0908(df,df2,"외교/통일/국방")
t_9 = pd.concat([temp_10,tm_9])
t_9.to_csv("외교_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
12,454149,외교/통일/국방,"제주도 불법 난민 신청 문제에 따른 난민법, 무사증 입국, 난민신청허가 폐지/개헌 ...",2018-07-13,714875


12   2018-05-13
Name: Date, dtype: datetime64[ns]

12   2018-06-12
Name: Date, dtype: datetime64[ns]

12   2018-07-13
Name: Date, dtype: datetime64[ns]

12   2018-08-13
Name: Date, dtype: datetime64[ns]

...


외교/통일/국방 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 71건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,제주도,예멘,제주,한국,청와대,청원,법무부,신청자,난민법,외국인,박상기,예멘인,무사증,우리나라,이슬람
1,48,42,39,30,26,24,22,22,20,16,15,14,13,10,9


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,예멘인들,일자리,유럽,미국,난민협약,서울,국민청원,일본,사람들,난민,가입국,천주교,제주교구,전염병,이주민
1,9,9,9,9,8,7,6,6,6,6,6,6,6,5,5


,0,1,기간
0,제주도,48,뉴스 C기간
1,예멘,42,뉴스 C기간
2,제주,39,뉴스 C기간
3,한국,30,뉴스 C기간
4,청와대,26,뉴스 C기간
5,청원,24,뉴스 C기간
6,법무부,22,뉴스 C기간
7,신청자,22,뉴스 C기간
8,난민법,20,뉴스 C기간
9,외국인,16,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,제주,2
1,뉴스 A기간,예멘,2
2,뉴스 A기간,무사증,2
3,뉴스 A기간,법무부,2
4,뉴스 A기간,외국인,2
...,...,...,...
25,뉴스 C기간,가입국,6
26,뉴스 C기간,천주교,6
27,뉴스 C기간,제주교구,6
28,뉴스 C기간,전염병,5


In [54]:
# 육아/교육
df2 = pd.read_excel('news_0908/육아_교육_20200218-20200520.xlsx')
tm_10 =wc_0908(df,df2,"육아/교육")
t_10 = pd.concat([temp_11,tm_10])
t_10.to_csv("육아_교육_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
20,454141,육아/교육,저희 25개월딸이 초등학생 5학년에게 성폭행을 당했습니다,2020-04-19,533883


20   2020-02-18
Name: Date, dtype: datetime64[ns]

20   2020-03-19
Name: Date, dtype: datetime64[ns]

20   2020-04-19
Name: Date, dtype: datetime64[ns]

20   2020-05-20
Name: Date, dtype: datetime64[ns]

...


육아/교육 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 29건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,청원,청와대,성폭행,국민청원,센터장,게시판,강정수,53만,청원인,디지털소통센터장,경찰청,성폭력,가해자,경기남,성범죄
1,24,22,22,17,16,14,13,12,10,9,9,7,7,7,7


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,50만,경찰,성추행,평택시,피해자,청원글,허위,개월,남학생,학생,사실,다음날,평택,20만,성관계
1,6,6,6,6,6,5,5,5,5,5,5,5,5,4,4


,0,1,기간
0,청원,24,뉴스 C기간
1,청와대,22,뉴스 C기간
2,성폭행,22,뉴스 C기간
3,국민청원,17,뉴스 C기간
4,센터장,16,뉴스 C기간
5,게시판,14,뉴스 C기간
6,강정수,13,뉴스 C기간
7,53만,12,뉴스 C기간
8,청원인,10,뉴스 C기간
9,디지털소통센터장,9,뉴스 C기간


,Source,Target,Weight
0,뉴스 B기간,피해자,8.0
1,뉴스 B기간,가해자,6.0
2,뉴스 B기간,성폭행,5.0
3,뉴스 B기간,성착취,5.0
4,뉴스 B기간,청원,4.0
5,뉴스 B기간,청와대,4.0
6,뉴스 B기간,가능성,3.0
7,뉴스 B기간,성범죄,3.0
8,뉴스 B기간,음란물,3.0
9,뉴스 B기간,n번방,3.0


In [34]:
# 인권/성평등
df2 = pd.read_excel('news_0908/인권_성평등_20200122-20200423.xlsx')
tm_11 =wc_0908(df,df2,"인권/성평등")
t_11 = pd.concat([temp_12,tm_11])
t_11.to_csv("인권_성평등_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
5,454156,인권/성평등,"신천지 예수교 증거장막성전(이하, 신천지)의 강제 해체(해산)을 청원합니다.",2020-03-23,1449521


5   2020-01-22
Name: Date, dtype: datetime64[ns]

5   2020-02-21
Name: Date, dtype: datetime64[ns]

5   2020-03-23
Name: Date, dtype: datetime64[ns]

5   2020-04-23
Name: Date, dtype: datetime64[ns]

...


인권/성평등 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 56건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,중국,코로나19,확진자,신천지,코로나,한국,대구,미국,유럽,코로,사망자,경북,문재인,감염자,감염병
1,39,35,35,32,26,25,24,19,15,14,10,9,9,9,8


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,입국자,시민들,중국발,의료진,1만,치명률,집단감염,페이스북,전문가,이탈리아,자가격리,스페인,황교안,개신교,신도
1,8,7,6,6,6,6,6,5,5,5,5,5,5,4,4


,0,1,기간
0,중국,39,뉴스 C기간
1,코로나19,35,뉴스 C기간
2,확진자,35,뉴스 C기간
3,신천지,32,뉴스 C기간
4,코로나,26,뉴스 C기간
5,한국,25,뉴스 C기간
6,대구,24,뉴스 C기간
7,미국,19,뉴스 C기간
8,유럽,15,뉴스 C기간
9,코로,14,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,확진자,41
1,뉴스 A기간,신천지,39
2,뉴스 A기간,코로나19,37
3,뉴스 A기간,대구,32
4,뉴스 A기간,중국,31
...,...,...,...
25,뉴스 C기간,자가격리,5
26,뉴스 C기간,스페인,5
27,뉴스 C기간,황교안,5
28,뉴스 C기간,개신교,4


In [35]:
# 일자리
df2 = pd.read_excel('news_0908/일자리_20200126-20200427.xlsx')
tm_12 =wc_0908(df,df2,"일자리")
t_12 = pd.concat([temp_13,tm_12])
t_12.to_csv("일자리_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
49,454112,일자리,한전 사업에 중국 기업의 참여를 허락하는 것은 말도 안됩니다.,2020-03-27,383039


49   2020-01-26
Name: Date, dtype: datetime64[ns]

49   2020-02-25
Name: Date, dtype: datetime64[ns]

49   2020-03-27
Name: Date, dtype: datetime64[ns]

49   2020-04-27
Name: Date, dtype: datetime64[ns]

...


일자리 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 3210건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,문재인,민주당,코로나19,청와대,위원장,통합당,더불어민주당,서울,코로나,한국,미래통합당,코로,지원금,김종인,황교안
1,1722,1140,1058,793,732,644,635,634,473,470,412,400,376,349,340


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,지역구,미국,관계자,대통령,코로나바이러스,더불어시민당,유권자,이낙연,국회의원,정의당,국민들,부산,대한민국,일자리,긴급재난지원금
1,327,310,294,268,264,258,248,245,233,232,222,221,221,205,202


,0,1,기간
0,문재인,1722,뉴스 C기간
1,민주당,1140,뉴스 C기간
2,코로나19,1058,뉴스 C기간
3,청와대,793,뉴스 C기간
4,위원장,732,뉴스 C기간
5,통합당,644,뉴스 C기간
6,더불어민주당,635,뉴스 C기간
7,서울,634,뉴스 C기간
8,코로나,473,뉴스 C기간
9,한국,470,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,문재인,1496
1,뉴스 A기간,청와대,1005
2,뉴스 A기간,서울,606
3,뉴스 A기간,중국,539
4,뉴스 A기간,한국당,522
...,...,...,...
25,뉴스 C기간,국민들,222
26,뉴스 C기간,부산,221
27,뉴스 C기간,대한민국,221
28,뉴스 C기간,일자리,205


In [36]:
# 정치개혁
df2 = pd.read_excel('news_0908/정치개혁_20190322-20190622.xlsx')
tm_13= wc_0908(df,df2,"정치개혁")
t_13 = pd.concat([temp_14,tm_13])
t_13.to_csv("정치개혁_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
2,454159,정치개혁,자유 한국당 정당해산 청원,2019-05-22,1831900


2   2019-03-22
Name: Date, dtype: datetime64[ns]

2   2019-04-21
Name: Date, dtype: datetime64[ns]

2   2019-05-22
Name: Date, dtype: datetime64[ns]

2   2019-06-22
Name: Date, dtype: datetime64[ns]

...


정치개혁 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 71건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,한국당,청와대,자유한국당,청원,민주당,나경원,강기정,원내대표,더불어민주당,황교안,국회의원,막말,문재인,패스트트랙,국민청원
1,62,46,36,33,27,26,25,22,21,19,18,18,17,16,14


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,국민들,바른미래당,주권자,비서관,대변인,정무수석,복기왕,선거법,이인영,민주주의,정의당,게시판,관계자,김무성,구속영장
1,14,14,14,11,10,10,10,10,9,9,9,9,8,8,7


,0,1,기간
0,한국당,62,뉴스 C기간
1,청와대,46,뉴스 C기간
2,자유한국당,36,뉴스 C기간
3,청원,33,뉴스 C기간
4,민주당,27,뉴스 C기간
5,나경원,26,뉴스 C기간
6,강기정,25,뉴스 C기간
7,원내대표,22,뉴스 C기간
8,더불어민주당,21,뉴스 C기간
9,황교안,19,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,한국당,19
1,뉴스 A기간,통영,13
2,뉴스 A기간,황교안,12
3,뉴스 A기간,고성,12
4,뉴스 A기간,경남,12
...,...,...,...
25,뉴스 C기간,정의당,9
26,뉴스 C기간,게시판,9
27,뉴스 C기간,관계자,8
28,뉴스 C기간,김무성,8


In [37]:
# 행정
df2 = pd.read_excel('news_0908/행정_20200607-20200907.xlsx')
tm_14 =wc_0908(df,df2,"행정")
t_14 = pd.concat([temp_15,tm_14])
t_14.to_csv("행정_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
34,454127,행정,정부의 교회 정규 예배 이외 행사 금지를 취소해주세요.,2020-08-07,427470


34   2020-06-07
Name: Date, dtype: datetime64[ns]

34   2020-07-07
Name: Date, dtype: datetime64[ns]

34   2020-08-07
Name: Date, dtype: datetime64[ns]

34   2020-09-07
Name: Date, dtype: datetime64[ns]

...


행정 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 846건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,확진자,코로나19,서울,코로나,수도권,2단계,사랑제일교회,서울시,제일교회,광화문,인천,경기도,행정명령,광복절,종교시설
1,471,461,407,234,201,177,162,160,158,128,116,116,102,100,100


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,2주,신도,집단감염,3단계,감염병,영상편집,코로나바이러스,온라인,방역당국,부산,관계자,방역수칙,전광훈,성북구,대구
1,94,92,92,92,91,87,83,81,79,78,76,72,62,61,60


,0,1,기간
0,확진자,471,뉴스 C기간
1,코로나19,461,뉴스 C기간
2,서울,407,뉴스 C기간
3,코로나,234,뉴스 C기간
4,수도권,201,뉴스 C기간
5,2단계,177,뉴스 C기간
6,사랑제일교회,162,뉴스 C기간
7,서울시,160,뉴스 C기간
8,제일교회,158,뉴스 C기간
9,광화문,128,뉴스 C기간


,Source,Target,Weight
0,뉴스 A기간,코로나19,107
1,뉴스 A기간,확진자,96
2,뉴스 A기간,서울,56
3,뉴스 A기간,코로나,51
4,뉴스 A기간,광주,48
...,...,...,...
25,뉴스 C기간,관계자,76
26,뉴스 C기간,방역수칙,72
27,뉴스 C기간,전광훈,62
28,뉴스 C기간,성북구,61


In [55]:
# 교통
df2 = pd.read_excel('news_0908/교통_건축_국토_20210226-20210529.xlsx')
tm_교통 =wc_0908(df,df2,"교통/건축/국토")
t_교통 = pd.concat([temp_교통,tm_교통])
t_교통.to_csv("교통_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
13,454148,교통/건축/국토,강원도 차이나타운 건설을 철회해주세요.,2021-04-28,670780


13   2021-02-26
Name: Date, dtype: datetime64[ns]

13   2021-03-28
Name: Date, dtype: datetime64[ns]

13   2021-04-28
Name: Date, dtype: datetime64[ns]

13   2021-05-29
Name: Date, dtype: datetime64[ns]

...


교통/건축/국토 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 10건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,차이나타운,강원도,한중문화타운,청와대,청원,한국,중국,국민청원,국민,지자체,가능성,사업,조성,반대,센터
1,10,10,7,6,6,6,5,4,4,3,3,3,3,3,3


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,동의,67만,우수성,수석실,디지털소통센터,소통,관광,건설,우수,문화,기여,가능,디지털,단서,목적
1,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2


,0,1,기간
0,차이나타운,10,뉴스 C기간
1,강원도,10,뉴스 C기간
2,한중문화타운,7,뉴스 C기간
3,청와대,6,뉴스 C기간
4,청원,6,뉴스 C기간
5,한국,6,뉴스 C기간
6,중국,5,뉴스 C기간
7,국민청원,4,뉴스 C기간
8,국민,4,뉴스 C기간
9,지자체,3,뉴스 C기간


,Source,Target,Weight
0,뉴스 B기간,강원도,43.0
1,뉴스 B기간,차이나타운,42.0
2,뉴스 B기간,중국,38.0
3,뉴스 B기간,한중문화타운,36.0
4,뉴스 B기간,청원,33.0
5,뉴스 B기간,청와대,30.0
6,뉴스 B기간,최문순,25.0
7,뉴스 B기간,강원,22.0
8,뉴스 B기간,한국,19.0
9,뉴스 B기간,국민청원,18.0


In [65]:
# 농어촌
df2 = pd.read_excel('news_0908/농산어촌_20190702-20191011.xlsx')
tm_농어촌 =wc_0908(df,df2,"농산어촌")
t_농어촌 = pd.concat([temp_농어촌,tm_농어촌])
t_농어촌.to_csv("농산어촌_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
1069,453089,농산어촌,왜 어민이 농민보다 세금을 더 내야 합니까?,2019-09-01,28310


1069   2019-07-02
Name: Date, dtype: datetime64[ns]

1069   2019-08-01
Name: Date, dtype: datetime64[ns]

1069   2019-09-01
Name: Date, dtype: datetime64[ns]

1069   2019-10-02
Name: Date, dtype: datetime64[ns]

...

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,수협중앙회,소득세,매출액,고령화,수산물,10억,재배업,수협,작물재배업,관계자,65세,3000만,해양수산부,내수면,축산업
1,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1


,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,청원,청와대,조합장,어민들,국민청원,홈페이지,조합장들,어업소득,총인구,11만,통계청,특용작물,수산업,식량작물,사람들
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


,0,1,기간
0,수협중앙회,2,뉴스 B기간
1,소득세,2,뉴스 B기간
2,매출액,2,뉴스 B기간
3,고령화,2,뉴스 B기간
4,수산물,2,뉴스 B기간
5,10억,2,뉴스 B기간
6,재배업,2,뉴스 B기간
7,수협,2,뉴스 B기간
8,작물재배업,1,뉴스 B기간
9,관계자,1,뉴스 B기간



농산어촌 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 0건 워드클라우드



,Source,Target,Weight
0,뉴스 B기간,수협중앙회,2.0
1,뉴스 B기간,소득세,2.0
2,뉴스 B기간,매출액,2.0
3,뉴스 B기간,고령화,2.0
4,뉴스 B기간,수산물,2.0
5,뉴스 B기간,10억,2.0
6,뉴스 B기간,재배업,2.0
7,뉴스 B기간,수협,2.0
8,뉴스 B기간,작물재배업,1.0
9,뉴스 B기간,관계자,1.0


In [66]:
# 저출산
df2 = pd.read_excel('news_0908/저출산_고령화대책_20201206-20210308.xlsx')
tm_저출산 =wc_0908(df,df2,"저출산/고령화대책")
t_저출산 = pd.concat([temp_저출산,tm_저출산])
t_저출산.to_csv("저출산_sns_news.csv",encoding = "cp949",index = False)

,Num,Class,Title,Date,Support
1136,453021,저출산/고령화대책,서울시임신출산정보센터 논란글 담당작성자와 책임자 징계 및 공개 사과를 요구합니다.,2021-02-05,26068


1136   2020-12-06
Name: Date, dtype: datetime64[ns]

1136   2021-01-05
Name: Date, dtype: datetime64[ns]

1136   2021-02-05
Name: Date, dtype: datetime64[ns]

1136   2021-03-08
Name: Date, dtype: datetime64[ns]

...


저출산/고령화대책 분야의 청원 완료일 +1일 부터 +30일까지(기간C) 1건 워드클라우드



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,임신부,사진전,은정,홈쇼핑,여성들,지지자,서울,미영,운영자,서울시,사진적,아이들,안선영,사진들,장국영
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


""
0
1


,0,1,기간
0,임신부,1,뉴스 C기간
1,사진전,1,뉴스 C기간
2,은정,1,뉴스 C기간
3,홈쇼핑,1,뉴스 C기간
4,여성들,1,뉴스 C기간
5,지지자,1,뉴스 C기간
6,서울,1,뉴스 C기간
7,미영,1,뉴스 C기간
8,운영자,1,뉴스 C기간
9,서울시,1,뉴스 C기간


,Source,Target,Weight
0,뉴스 B기간,서울시,5.0
1,뉴스 B기간,홈페이지,4.0
2,뉴스 B기간,가족들,4.0
3,뉴스 B기간,성차별적,3.0
4,뉴스 B기간,아이사랑,3.0
5,뉴스 B기간,밑반찬,3.0
6,뉴스 B기간,임신출산정보센터,2.0
7,뉴스 B기간,뭇매,2.0
8,뉴스 B기간,임신부,2.0
9,뉴스 B기간,아이들,2.0
